# Description

In [60]:
import pandas as pd
# Reading the dataset
df=pd.read_json(r'C:\Users\pavan\Downloads\databricks-dolly-15k.jsonl\databricks-dolly-15k.jsonl',lines=True)

In [61]:
df

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,closed_qa
1,Which is a species of fish? Tope or Rope,,Tope,classification
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,open_qa
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,open_qa
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",closed_qa
...,...,...,...,...
15010,How do i accept the change,,Embrace the change and see the difference,brainstorming
15011,What is a laser and who created it?,A laser is a device that emits light through a...,A laser is a device that emits light from an e...,summarization
15012,What is the difference between a road bike and...,,Road bikes are built to be ridden on asphalt a...,open_qa
15013,How does GIS help in the real estate investmen...,,"Real estate investors depend on precise, accur...",general_qa


In [62]:
# Observing the any 10 samples from the dataframe.
df.sample(10)


,instruction,context,response,category
14215,Summarize the lay out of the Jardins du Châtea...,The Jardins du Château Val Joanis are gardens ...,The Jardins du Château Val Joanis are separate...,summarization
9913,In what year was pickleball invented?,The game was created in 1965 on Bainbridge Isl...,Pickleball was created in 1965 on Bainbridge I...,closed_qa
4348,What is an office space?,,An office space is a room or group of rooms in...,open_qa
7713,Tell me about the play Die Schutzbefohlenen,"Die Schutzbefohlenen (translated as ""Charges (...","Die Schutzbefohlenen, translated as the suppli...",closed_qa
2113,What is an index fund?,,An index fund is a basket of individual stocks...,open_qa
9388,Classify the following foods by macronutrient:...,,The three types of macronutrient are carbohydr...,classification
9548,What is a fun and cheap activity you can do in...,,In the summer it is nice to go to St James par...,general_qa
12047,Can you summarize for me in no more than three...,The Little Engine That Could is an American fo...,The Little Engine That Could is a children's b...,summarization
12668,"Extract the names of the noble award winners, ...",Five people have received two Nobel Prizes. Ma...,•\tMarie Curie - Physics - 1903\n•\tMarie Curi...,information_extraction
1255,What is it like to live in a black hole?,,I am the first person to survive a black hole ...,creative_writing


In [42]:
# importing the preprocessing package, nltk and its related modules.
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
      opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])


Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [63]:
# Let's create a dictionary which we can use for scoring our review text.
nltk.download('punkt')
df.rename(columns={"response": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
 
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
        word_dict[word] = pos_score
      
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
        word_dict[word] = neg_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
# Analysing the sentiment score.
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score  


In [65]:
# Replacing null values with no review.
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)


In [66]:
# observing the first 10 rows for the columns overall,text and Bing_Liu_Score. 
df[['instruction',"text", 'Bing_Liu_Score']].head(10)


,instruction,text,Bing_Liu_Score
0,When did Virgin Australia start operating?,Virgin Australia commenced services on 31 Augu...,0
1,Which is a species of fish? Tope or Rope,Tope,0
2,Why can camels survive for long without water?,Camels use the fat in their humps to keep them...,-1
3,"Alice's parents have three daughters: Amy, Jes...",The name of the third daughter is Alice,0
4,When was Tomoaki Komorida born?,"Tomoaki Komorida was born on July 10,1981.",0
5,If I have more pieces at the time of stalemate...,No. \nStalemate is a drawn position. It doesn'...,0
6,"Given a reference text about Lollapalooza, whe...",Lollapalooze is an annual musical festival hel...,0
7,Who gave the UN the land in NY to build their HQ,John D Rockerfeller,0
8,Why mobile is bad for human,We are always engaged one phone which is not g...,1
9,Who was John Moses Browning?,John Moses Browning is one of the most well-kn...,7


In [67]:
# finding mean for the column Bing_Liu_Score by performing groupby on overall.
df.groupby('instruction').agg({'Bing_Liu_Score':'mean'})


,Bing_Liu_Score
instruction,
"""Big Bang"": what does it mean?",-2.0
"""The Day the Music Died"" refers to the tragic deaths of which artists?",0.0
"""We can do it"" slogan / catchphrase - Who made it?",0.0
"""William Bowery"" is a pseudonym for whom?",0.0
"""repair instead of replace"" give five points to support this argument",2.0
...,...
write short essay on Buzz Aldrin experience on moon,-2.0
write short essay on Neil Armstrong experience on moon,10.0
write some critical points on University of Sfax using below text as a reference,0.0
